In [27]:
import os
import re
import json
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:,.2f}'.format)

class UpbitMicrostructure:
    """업비트 거래 환경 정밀 모사"""
    FEE_RATE = 0.0005  # 0.05%
    MIN_ORDER_KRW = 5000
    
    @staticmethod
    def get_tick_size(price):
        if price >= 2_000_000: return 1000
        elif price >= 1_000_000: return 500
        elif price >= 500_000: return 100
        elif price >= 100_000: return 50
        elif price >= 10_000: return 10
        elif price >= 1_000: return 5
        elif price >= 100: return 1
        elif price >= 10: return 0.1
        else: return 0.01

    @staticmethod
    def adjust_to_tick(price):
        tick = UpbitMicrostructure.get_tick_size(price)
        return (price // tick) * tick

    @staticmethod
    def apply_slippage(price, is_buying=True, ticks=1):
        adjusted_price = price
        for _ in range(ticks):
            tick = UpbitMicrostructure.get_tick_size(adjusted_price)
            if is_buying: adjusted_price += tick
            else: 
                adjusted_price -= tick
                if adjusted_price < 0: adjusted_price = 0
        return adjusted_price

class QuantSimulator:
    def __init__(self, initial_capital=1_000_000):
        self.env = UpbitMicrostructure()
        self.initial_capital = initial_capital

    def calculate_metrics(self, equity_curve, dates, trades_log):
        if len(equity_curve) < 2: 
            return {'CAGR': 0, 'MDD': 0, 'Sharpe': 0, 'WinRate': 0, 'PF': 0}
            
        equity_s = pd.Series(equity_curve, index=dates)
        
        total_days = (dates[-1] - dates[0]).days
        if total_days < 1: total_days = 1
        total_ret = (equity_s.iloc[-1] / equity_s.iloc[0]) - 1
        cagr = ((1 + total_ret) ** (365 / total_days)) - 1
        
        roll_max = equity_s.cummax()
        drawdown = (equity_s - roll_max) / roll_max
        mdd = drawdown.min() * 100
        
        daily_eq = equity_s.resample('D').last().ffill()
        daily_ret = daily_eq.pct_change().dropna()
        rf_daily = 0.035 / 365
        
        sharpe = 0.0
        if len(daily_ret) > 1 and daily_ret.std() != 0:
            sharpe = (daily_ret.mean() - rf_daily) / daily_ret.std() * np.sqrt(365)

        wins = [t for t in trades_log if t > 0]
        losses = [t for t in trades_log if t <= 0]
        
        win_rate = (len(wins) / len(trades_log) * 100) if trades_log else 0
        gross_profit = sum(wins)
        gross_loss = abs(sum(losses))
        profit_factor = (gross_profit / gross_loss) if gross_loss > 0 else 99.99
        
        return {
            'CAGR': round(cagr * 100, 2),
            'MDD': round(mdd, 2),
            'Sharpe': round(sharpe, 2),
            'WinRate': round(win_rate, 2),
            'PF': round(profit_factor, 2)
        }

    def run_simulation_detail(self, df, df_hour, start_capital, model_threshold):
        equity = start_capital
        equity_curve = [equity]
        dates = [pd.to_datetime(df['date'].iloc[0])]
        trades_log = []
        
        df = df.sort_values('date').reset_index(drop=True)
        # df_hour는 외부에서 정렬된 상태로 받는다고 가정
        
        start_p = self.env.apply_slippage(df['entry_price'].iloc[0], True)
        end_p = self.env.apply_slippage(df['entry_price'].iloc[-1], False)
        bh_ret = (end_p - start_p) / start_p * 100
        
        # === 자금 관리 설정 ===
        VOL_TARGET = 0.02
        KELLY_FRACTION = 0.5
        # [핵심] metadata.json에서 가져온 Threshold 적용
        MIN_PROB = model_threshold 
        
        for date_val, daily_group in df.groupby('date'):
            # Threshold 이상인 신호만 필터링
            candidates = daily_group[daily_group['prob'] >= MIN_PROB]
            
            if candidates.empty:
                equity_curve.append(equity)
                dates.append(date_val)
                continue
            
            # 가장 확률 높은 1개 선택
            best_signal = candidates.sort_values('prob', ascending=False).head(1)
            row = best_signal.iloc[0]
            
            entry_p = self.env.apply_slippage(self.env.adjust_to_tick(row['entry_price']), True)
            atr_val = row['atr'] if row['atr'] > 0 else entry_p * 0.01
            
            # --- Position Sizing ---
            vol_qty = (equity * VOL_TARGET) / atr_val
            model_prob = row['prob']
            potential_reward = row['tp_price'] - entry_p
            potential_risk = entry_p - row['sl_price']
            
            if potential_risk <= 0: potential_risk = entry_p * 0.005
            risk_reward_ratio = potential_reward / potential_risk
            if risk_reward_ratio <= 0: risk_reward_ratio = 1.0
            
            kelly_f = model_prob - ((1 - model_prob) / risk_reward_ratio)
            
            if kelly_f <= 0:
                equity_curve.append(equity)
                dates.append(date_val)
                continue
                
            kelly_risk_amt = equity * kelly_f * KELLY_FRACTION
            kelly_qty = kelly_risk_amt / potential_risk
            qty = min(vol_qty, kelly_qty)
            max_qty_cash = (equity * 0.99) / entry_p
            qty = min(qty, max_qty_cash)
            
            pos_krw = qty * entry_p
            if pos_krw < self.env.MIN_ORDER_KRW: 
                equity_curve.append(equity)
                dates.append(date_val)
                continue
            
            fee_buy = pos_krw * self.env.FEE_RATE
            equity -= fee_buy
            
            tp_price = row['tp_price']
            sl_price = row['sl_price']
            
            entry_time = date_val + pd.Timedelta(hours=9)
            end_time = entry_time + pd.Timedelta(days=1)
            
            # 1시간봉 데이터 조회 (최적화 필요시 슬라이싱 방식 개선 가능)
            hourly_data = df_hour[(df_hour['datetime'] >= entry_time) & (df_hour['datetime'] < end_time)]
            
            exit_raw = hourly_data.iloc[-1]['close'] if not hourly_data.empty else row['entry_price']
            exit_p = self.env.apply_slippage(self.env.adjust_to_tick(exit_raw), False)
            
            triggered = False
            for _, h_row in hourly_data.iterrows():
                if h_row['low'] <= sl_price:
                    exit_p = self.env.apply_slippage(self.env.adjust_to_tick(sl_price), False)
                    triggered = True
                    break 
                if h_row['high'] >= tp_price:
                    exit_p = self.env.apply_slippage(self.env.adjust_to_tick(tp_price), False)
                    triggered = True
                    break
            
            sell_amt = qty * exit_p
            fee_sell = sell_amt * self.env.FEE_RATE
            
            net_pnl = sell_amt - fee_sell - pos_krw
            equity += net_pnl
            trades_log.append(net_pnl)
            
            equity_curve.append(equity)
            dates.append(date_val)
            
        metrics = self.calculate_metrics(equity_curve, dates, trades_log)
        metrics['End_Eq'] = equity
        metrics['BH_Ret'] = round(bh_ret, 2)
        metrics['My_Ret'] = round((equity - start_capital) / start_capital * 100, 2)
        metrics['Trades'] = len(trades_log)
        metrics['Start_Date'] = df['date'].min().strftime('%Y-%m-%d')
        metrics['End_Date'] = df['date'].max().strftime('%Y-%m-%d')
        metrics['Threshold_Used'] = MIN_PROB
        
        return metrics

class FullMetricTester:
    def __init__(self, root_dir, hourly_data_path):
        self.root_dir = root_dir
        self.sim = QuantSimulator(initial_capital=1_000_000) 
        self.data_map = {}
        self.candidates = [] 
        
        print(f"📂 Loading Hourly Data from {hourly_data_path}...")
        try:
            self.df_hour = pd.read_csv(hourly_data_path)
            self.df_hour['datetime'] = pd.to_datetime(self.df_hour['datetime'])
            self.df_hour = self.df_hour.sort_values('datetime').reset_index(drop=True)
            print("✅ Hourly Data Loaded Successfully.")
        except Exception as e:
            print(f"❌ Failed to load hourly data: {e}")
            self.df_hour = pd.DataFrame()

    def collect(self):
        print(f"📂 Scanning all trials in: {self.root_dir}")
        # os.listdir로 trial 폴더 탐색
        trial_dirs = [os.path.join(self.root_dir, d) for d in os.listdir(self.root_dir) if os.path.isdir(os.path.join(self.root_dir, d)) and 'trial_' in d]
        trial_dirs.sort(key=lambda x: int(re.search(r'trial_(\d+)_', x).group(1)) if re.search(r'trial_(\d+)_', x) else 0)
        
        count = 0
        loaded_meta = 0
        
        for trial_path in trial_dirs:
            trial_name = os.path.basename(trial_path)
            t_match = re.search(r'trial_(\d+)_', trial_name)
            if not t_match: continue
            trial_id = int(t_match.group(1))
            
            # 각 Trial 내의 Fold 폴더 순회
            for root, dirs, files in os.walk(trial_path):
                pred_files = [f for f in files if f.startswith('predictions_') and f.endswith('.csv')]
                if not pred_files: continue
                
                fold_name = os.path.basename(root)
                
                # [중요] metadata.json 위치: root(fold폴더)/metadata.json
                meta_path = os.path.join(root, 'metadata.json')
                best_thresholds = {}
                
                if os.path.exists(meta_path):
                    try:
                        with open(meta_path, 'r') as f:
                            meta_data = json.load(f)
                            best_thresholds = meta_data.get('best_thresholds', {})
                            loaded_meta += 1
                    except:
                        pass
                
                for f in pred_files:
                    m_name = f.replace('predictions_', '').replace('.csv', '')
                    df = pd.read_csv(os.path.join(root, f))
                    df['date'] = pd.to_datetime(df['date'])
                    
                    
                    th_val = best_thresholds.get(m_name, 0.5)
                    print(th_val)
                    
                    key = (trial_id, trial_name, m_name)
                    if key not in self.data_map: self.data_map[key] = []
                    if any(item['fold'] == fold_name for item in self.data_map[key]): continue
                    
                    self.data_map[key].append({'fold': fold_name, 'df': df, 'threshold': th_val})
                    count += 1
        print(f"✅ Collected {count} prediction files. (Metadata found in {loaded_meta} folds)")

    def _simulate_single_model(self, key, folds):
        t_id, t_name, m_name = key
        folds.sort(key=lambda x: x['df']['date'].min())
        
        wf_folds = [f for f in folds if 'final' not in f['fold']]
        final_folds = [f for f in folds if 'final' in f['fold']]
        
        # 1. Walk-Forward Simulation (누적 자본금 적용)
        wf_equity = 1_000_000
        wf_trades = 0
        wf_details = []
        
        for item in wf_folds:
            res = self.sim.run_simulation_detail(item['df'], self.df_hour, wf_equity, item['threshold'])
            wf_equity = res['End_Eq']
            wf_trades += res['Trades']
            wf_details.append(res)
            
        wf_ret = (wf_equity - 1_000_000) / 1_000_000 * 100
        
        # 2. Final Holdout Simulation (독립 수행)
        final_equity = 1_000_000
        final_trades = 0
        final_details = []
        
        for item in final_folds:
            res = self.sim.run_simulation_detail(item['df'], self.df_hour, final_equity, item['threshold'])
            final_equity = res['End_Eq']
            final_trades += res['Trades']
            final_details.append(res)
            
        final_ret = (final_equity - 1_000_000) / 1_000_000 * 100
        
        # 승률 계산 (가중 평균)
        wf_wins = sum([d['WinRate'] * d['Trades'] for d in wf_details]) / wf_trades if wf_trades > 0 else 0
        final_wins = sum([d['WinRate'] * d['Trades'] for d in final_details]) / final_trades if final_trades > 0 else 0

        return {
            'key': key,
            'WF_Return': wf_ret,
            'Final_Return': final_ret,
            'WF_Trades': wf_trades,
            'Final_Trades': final_trades,
            'WF_WinRate': wf_wins,
            'Final_WinRate': final_wins,
            'WF_Details': wf_details,
            'Final_Details': final_details
        }

    def run(self):
        if self.df_hour.empty:
            print("⚠️ Warning: Hourly data is empty.")
            
        print("\n🚀 Simulating all models... This may take a while.")
        
        for key, folds in self.data_map.items():
            result = self._simulate_single_model(key, folds)
            self.candidates.append(result)
            
        best_wf_model = max(self.candidates, key=lambda x: x['WF_Return'])
        best_final_model = max(self.candidates, key=lambda x: x['Final_Return'])
        
        self._print_champion_report("🏆 BEST WALK-FORWARD MODEL", best_wf_model)
        self._print_champion_report("🌟 BEST FINAL-HOLDOUT MODEL", best_final_model)

    def _print_champion_report(self, title, model_data):
        t_id, t_name, m_name = model_data['key']
        print(f"\n{'='*100}")
        print(f"{title}")
        print(f"Trial: {t_name} | Model: {m_name}")
        print(f"Walk-Forward Return: {model_data['WF_Return']:,.1f}% ({model_data['WF_Trades']} trades, Win: {model_data['WF_WinRate']:.1f}%)")
        print(f"Final-Holdout Return: {model_data['Final_Return']:,.1f}% ({model_data['Final_Trades']} trades, Win: {model_data['Final_WinRate']:.1f}%)")
        print(f"{'='*100}")
        
        print(f"\n[Detailed Performance by Fold]")
        print(f"{'Period':<23} {'Type':<10} {'Return':>8}% {'MDD':>7}% {'Sharpe':>6} {'PF':>5} {'Win%':>5} {'Cnt':>3}")
        print("-" * 100)
        
        for res in model_data['WF_Details']:
            print(f"{res['Start_Date']}~{res['End_Date']:<12} {'WF':<10} {res['My_Ret']:>8.1f} {res['MDD']:>7.1f} {res['Sharpe']:>6.2f} {res['PF']:>5.1f} {res['WinRate']:>5.1f} {res['Trades']:>3}")
            
        for res in model_data['Final_Details']:
            print(f"{res['Start_Date']}~{res['End_Date']:<12} {'FINAL':<10} {res['My_Ret']:>8.1f} {res['MDD']:>7.1f} {res['Sharpe']:>6.2f} {res['PF']:>5.1f} {res['WinRate']:>5.1f} {res['Trades']:>3}")
        print("\n")

if __name__ == "__main__":
    base_path = "2025-11-26" 
    hourly_data_path = "../eth_hour.csv"
    
    if os.path.exists(base_path) and os.path.exists(hourly_data_path):
        tester = FullMetricTester(base_path, hourly_data_path)
        tester.collect()
        tester.run()
    else:
        print(f"Path check failed")


📂 Loading Hourly Data from ../eth_hour.csv...
✅ Hourly Data Loaded Successfully.
📂 Scanning all trials in: 2025-11-26
0.508024582547546
0.48248273520401336
0.4842256974110257
0.5085220291478979
0.565439763599491
0.4259922258874054
0.4226309715352906
0.5022041598761816
0.38937195988273865
0.20516122402469325
0.4186176944997249
0.42421100100132425
0.5346165293135363
0.5132229011601575
0.6031864600517023
0.6512016031227598
0.4979554882058211
0.2976190136779106
0.43269649280264333
0.4972557243611966
0.5061548323097703
0.32620897632112056
0.47501309115422474
0.4614144443729387
0.5473435739400423
0.38500518011425444
0.5026934053195058
0.591317442317245
0.5121261640654194
0.4162536163668669
0.4888814704974843
0.5897764717402335
0.49880529044844923
0.3223083378555444
0.5452716863390529
0.49726790288991396
0.5675301171393147
0.24662103723449144
0.4556964284641617
0.4944812077933227
0.39403266327203446
0.20040214760891217
0.37629556264742725
0.4916057212253407
0.5547680678784579
0.42982729206551

0.6567743004779839
0.5642114660235448
0.5112003675799116
0.34791723941220026
0.4721634971229666
0.5005259090057772
0.4117741971731783
0.3795483700367518
0.4651617211701126
0.5155190542359129
0.6463144497485681
0.4675818587129793
0.5118175744790991
0.5756442187218285
0.4395091753275919
0.3565348929399299
0.5055677605605042
0.5215505806558027
0.6551560074187242
0.42808268152629975
0.5283262666541486
0.48466077872528146
0.5888906431873402
0.2259398153337679
0.44908258108879795
0.5018390118775945
0.2575374140316501
0.21120407090004875
0.21140570213260407
0.41371529502521537
0.6515880219721352
0.4125388268978739
0.4891233941732355
0.5057934370600152
0.4865898779459679
0.2823039785310295
0.6121688724449609
0.569237057086738
0.5555427712886636
0.3581747160434096
0.5084841517424692
0.503796859290752
0.7099421997866774
0.47680134675626773
0.6802371218495745
0.6637348501315394
0.6806814129339578
0.2889839039376019
0.49672550071012583
0.5800861713062976
0.6221824083605184
0.4042952018242642
0.533

0.3182593918118882
0.23344972693105784
0.43443148400028997
0.41525393485292056
0.704166384171121
0.44458247495147546
0.7087727624244688
0.7059830486625189
0.4889397688566021
0.31534667348581363
0.4342855522979442
0.49839404574050666
0.43440663764259413
0.3622932237458256
0.4804061352416732
0.5022262418073207
0.6576612467379401
0.49931703162430796
0.509041885157233
0.518621573772932
0.6894372923587202
0.38290366639294227
0.5092885078849452
0.6037901067450288
0.5004935739107171
0.3233690475474996
0.6174297810581135
0.5255563417855755
0.4698829480143738
0.336019271507447
0.5334511436803108
0.5105735964683858
0.4378553442793538
0.2485259575062398
0.4855252064055839
0.4637633572622034
0.5972427515402464
0.44458776835178
0.5286388451716558
0.6106415638516219
0.5026472670526577
0.3093840850162197
0.42621981728054587
0.5010105011274352
0.4153883352272849
0.32291361781811867
0.5006730216870217
0.4730815987431809
0.49762221532473483
0.43290340566357827
0.4935197514371892
0.5592909686861098
0.560